In [8]:
import pynput
import HandTrackingModule as htm
import autopy
import time
import math
print("done")

done


In [7]:
import cv2
import mediapipe as mp
import pyautogui
import random
import numpy as np
from pynput.mouse import Button, Controller

mouse = Controller()
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

mute_icon = cv2.imread("mute.png", cv2.IMREAD_COLOR)
icon_resized = cv2.resize(mute_icon, (50, 50))
x_offset, y_offset = 50, 80


# Volume control initialization 
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume_control = cast(interface, POINTER(IAudioEndpointVolume))

vol_min, vol_max = volume_control.GetVolumeRange()[:2]

# Helper Functions
def get_angle(a, b, c):
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(np.degrees(radians))
    return angle

def get_distance(landmark_list):
    if len(landmark_list) < 2:
        return
    (x1, y1), (x2, y2) = landmark_list[0], landmark_list[1]
    L = np.hypot(x2 - x1, y2 - y1)
    return np.interp(L, [0, 1], [0, 1000])

screen_width, screen_height = pyautogui.size()


mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=2
)

# Gesture Detection for Right Hand
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        index_finger_tip = hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
        return index_finger_tip
    return None

def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y / 2 * screen_height)
        pyautogui.moveTo(x, y)

def is_left_click(landmark_list, thumb_index_dist):
    return (
        get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
        get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) > 90 and
        thumb_index_dist > 50
    )

def is_right_click(landmark_list, thumb_index_dist):
    return (
        get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
        get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90 and
        thumb_index_dist > 50
    )

def is_double_click(landmark_list, thumb_index_dist):
    return (
        get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
        get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
        thumb_index_dist > 50
    )

def is_screenshot(landmark_list, thumb_index_dist):
    return (
        get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
        get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
        thumb_index_dist < 50
    )

def is_volume_control(landmark_list):
    if len(landmark_list) >= 9:
        wrist = landmark_list[0]
        index_base = landmark_list[5]
        thumb_tip = landmark_list[4]
        index_tip = landmark_list[8]

        hand_size_ref = np.hypot(index_base[0] - wrist[0], index_base[1] - wrist[1])
        pinch_dist = np.hypot(index_tip[0] - thumb_tip[0], index_tip[1] - thumb_tip[1])

        if hand_size_ref == 0:
            return False
        ratio = pinch_dist / hand_size_ref
        return True
    return False

def fingers_up(landmarks):
    fingers = []

    # Index
    fingers.append(landmarks[8][1] < landmarks[6][1])
    # Middle
    fingers.append(landmarks[12][1] < landmarks[10][1])
    # Ring
    fingers.append(landmarks[16][1] < landmarks[14][1])
    # Pinky
    fingers.append(landmarks[20][1] < landmarks[18][1])
    return fingers



def perform_volume_control(frame, landmark_list, volume_control, vol_min, vol_max, mute_icon, x_offset, y_offset):
    wrist = landmark_list[0]
    index_base = landmark_list[5]
    thumb_tip = landmark_list[4]
    index_tip = landmark_list[8]

    hand_size_ref = np.hypot(index_base[0] - wrist[0], index_base[1] - wrist[1])
    pinch_dist = np.hypot(index_tip[0] - thumb_tip[0], index_tip[1] - thumb_tip[1])

    if hand_size_ref == 0:
        return

    ratio = pinch_dist / hand_size_ref
    cv2.putText(frame, f"Ratio: {ratio:.2f}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Draw volume bar background
    cv2.rectangle(frame, (50, 150), (85, 400), (0, 255, 0), 3)

    if ratio < 0.35:
        volume_control.SetMasterVolumeLevel(vol_min, None)
        frame[y_offset:y_offset+mute_icon.shape[0], x_offset:x_offset+mute_icon.shape[1]] = mute_icon
        cv2.putText(frame, "Muted", (50, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        volume = np.interp(ratio, [0.3, 0.9], [vol_min, vol_max])
        volume_control.SetMasterVolumeLevel(volume, None)

        # Volume bar fill
        vol_bar = np.interp(ratio, [0.3, 0.9], [400, 150])
        cv2.rectangle(frame, (50, int(vol_bar)), (85, 400), (0, 255, 0), cv2.FILLED)
        percent = np.interp(ratio, [0.3, 0.9], [0, 100])
        cv2.putText(frame, f'Volume: {int(percent)}%', (50, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)


def detect_gesture(frame, landmark_list, processed):
    if len(landmark_list) >= 21:
        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = get_distance([landmark_list[4], landmark_list[5]])

        if get_distance([landmark_list[4], landmark_list[5]]) < 50 and get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90:
            move_mouse(index_finger_tip)
        elif is_left_click(landmark_list,  thumb_index_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif is_right_click(landmark_list, thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        elif is_double_click(landmark_list, thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        elif is_screenshot(landmark_list, thumb_index_dist):
            im1 = pyautogui.screenshot()
            label = random.randint(1, 1000)
            im1.save(f'my_screenshot_{label}.png')
            cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

# Placeholder for Left Hand (you will define this)
def other_gesture(frame, landmark_list, processed):
    # Get relevant landmarks
    thumb_tip = landmark_list[4]
    thumb_ip = landmark_list[3]
    thumb_base  = landmark_list[2]
    # Detect which fingers are up
    fingers = fingers_up(landmark_list)

    # Trigger: all fingers up (i.e., open palm)
    if all(fingers):
        # Scroll mode activated
        distance = np.hypot(thumb_tip[0] - thumb_base[0], thumb_tip[1] - thumb_base[1]) * 1000

        if (thumb_tip[1] < thumb_ip[1] and distance>50):  # Thumb up
            pyautogui.scroll(50)
            cv2.putText(frame, "Scroll Up", (50, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 200, 0), 2)
        else:  # Thumb dwon
            pyautogui.scroll(-50)
            cv2.putText(frame, "Scroll Down", (50, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    else:
        # If not in scroll mode, allow volume control
        if is_volume_control(landmark_list):
            perform_volume_control(frame, landmark_list, volume_control, vol_min, vol_max, icon_resized, x_offset, y_offset)

       


def main():
    draw = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)

            if processed.multi_hand_landmarks and processed.multi_handedness:
                for idx, hand_label in enumerate(processed.multi_handedness):
                    hand_type = hand_label.classification[0].label  # 'Left' or 'Right'
                    hand_landmarks = processed.multi_hand_landmarks[idx]
                    draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)

                    # Get landmark list
                    landmark_list = [(lm.x, lm.y) for lm in hand_landmarks.landmark]

                    # Call appropriate function
                    if hand_type == 'Right':
                        detect_gesture(frame, landmark_list, processed)
                    elif hand_type == 'Left':
                        other_gesture(frame, landmark_list, processed)

            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()


if __name__ == '__main__':
    main()